# Overview (Example: Twitter)

## Requirement clarification

- Do users post tweets and follow other people?
- Do users search tweets?
- Need to display user timeline?
- Need to push notification on new tweets?
- Need to display trending topics?
- Do tweets contain photos and videos?
- Are we discussing backend only or frontend too?

## System estimation

- Numer of tweets? Number of users? 
- Size of storage?
- Network bandwidth?

## System interface

- What APIs are expected?

## Data model

- User (UserID, Name, Email, CreationDate, LastLogin, etc)
- Tweet (TweetID, Content, TimeStamp, etc)
- UserFollow (UserID1, UserID2)

## High-level design

- Multiple app servers to do read/write with LoadBalancer in front.
- Efficient DB to store all tweets and support large number of reads.
- Distributed file storage to store photos and videos.

## Detailed design

- How to partition data and distribute it to multiple DBs.
- How to handle "hot" users.
- At which layer should we introduce cache. 

## Identify bottleneck

- Is there a single point of failure?
- Do we have replicas of data?
- Do we have availability set of app servers?
- Do we monitor servers and get alerts?

# 5. Facebook messenger

## Functional

- Supports 1-on-1 conversation between users.
- Tracks online/offline status of users.
- Persists chat history.

## Non-functional

- Minimum latency when chatting.
- Consistency: same chat history from all devices.
- High availability.

## Extended

- Support group chats.
- Notify users of new messages when they are offline.

## Capacity

Assume 
- 500M daily active users.
- Each user sends 40 messages per day.
- 100 bytes per message.

Storage
- 500M * 40 messages = 20B messages per day.
- 20B messages per day * 100 bytes = 2TB per day.
- For 5 years, 2 TB * 365 days * 5 years = 3.6 PB

Bandwidth
- 2TB / 86400s = 25MB/s 
- Each message is going to other users, so need the same bandwidth for upload and download.

## Design

<img src="img/facebookmessenger1.png" style="width:500px;height:300px;">

When user A sends a message to user B.
- Server receives the message and send ack back to A.
- Server stores the message into DB and sends message to B.
- B receives the message and sends ack back to the server.
- Server notifies A that the message has been delivered.

How to handle message
- If users poll the servers to see if new messages, polling needs to be frequent to minimize latency for users and most polling would return empty response. 
- If active users keep connection with the server, users can receive messages as soon as servers receive messages.

How to keep connection
- HTTP long polling: servers hold client requests until there is some response to send back. If timeout, client must send new request.
- WebSockets
- If servers fail, users should reconnect.

How to track open connections
- Servers can use hash table where key is UserID and value is connection object.

How to handle offline user
- Servers can notify sender that delivery has failed.
- If receiver connects back, servers can ask sender to retry.

How many servers are needed
- Assume 500M connections and one server can handle 50k concurrent conecctions. Then, 10k servers are needed.

How to know which server has which user connection
- Use load balancers in front of server that map UserID to server.

Which DB to use
- RDBMS is not good because we need to do frequent read/write.
- Choose wide-column DB like HBase. HBase stores data into memory buffer and move data into disk when buffer is full. This works well with variable-sized data.

How to manage user status
- When users start the app, pull friends status from servers.
- When users start chatting with other users, pull status at that time.

<img src="img/facebookmessenger2.png" style="width:700px;height:500px;">

Design summary
- Clients open connection to servers to send messages.
- Servers send messages to requested users.
- All active clients keeps connection to servers.
- Messages are stored in HBase.
- Clients pull relevant user status from servers.

## Data paritioning

- Partition based on UserID hash.
- All messages of a user is stored in the same DB.

## Caching

- Cache recent messages in recent conversations.

## Load balancing

- In front of chat servers and cache servers.

## 

# Design Twitter

## Functional requirement

- Users post new tweets.
- Users follow other users.
- Users mark tweets as favs.
- Display users timeline with top tweets.
- Tweets contain photos and videos.

## Non-functional requirement

- High availability.
- 200 ms latency for timeline generation.

## Capacity estimate

- Assume 
    - 1B total users.
    - 200M daily active users.
    - 100M new tweets every day.
    - Each user follows 200 people.
    - Each user favs 5 tweeks per day.
- 200M * 5 favs = 1B favs per day.

## API

- tweet(api_dev_key, tweet_data, tweet_location, user_location, media_ids)

## High-level design

- Client -> load balancer -> app servers -> DB & file storage

## DB

Tweet
- TweetID (int, pk)
- UserID (int)
- Content (varchar)
- CreationDate (datetime)
- NumFavs (int)

User
- UserID (int, pk)
- Name (varchar)
- Email (varchar)
- DateOfBirth (datetime)
- CreationDate (datetime)
- LastLogin (datetime)

UserFollow
- UserID1 (int, pk)
- UserID2 (int, pk)

Favorite
- TweetID (int, pk)
- UserID (int, pk)
- CreationDate (datetime)

# Design Youtube

## Functional requirement

- Users upload videos.
- Users share and view videos.
- Users search videos.
- Tracks likes/dislikes, number of views.
- Users add and view comments.

## Non-functional requirement

- High availability.
- High reliability.
- Users should not feel lag watching videos.

## Capacity estimate

- Assume
    - 1.5B total users.
    - 800M daily active users.
    - Users watch 5 videos per day.
    - Ratio of upload:view is 1:200. 
- Videos view per second: 800M * 5 / 86400s = 46K videos/s.
- Videso upload per second: 46K / 200 = 230 videos/s.

## API

- uploadVideo(api_dev_key, video_title, video_description, tags[], category_id, default_language, recording_details, video_contents)
- searchVideo(api_dev_key, search_query, user_location, max_videos_to_return)
- streamVideo(api_dev_key, video_id, offset, codec, resolution)

## High-level design

- Client <-> web server <-> app server <-> DB / video storage.

## DB

# Design Typeahead Suggestion

## Functional requirement

- Suggests top 10 terms starting with whatever user has typed.

## Non-functional requirement

- Users see suggestions within 200ms.

# Design API Rate Limiter

- Prevents Dos attack, brute force password attempts, etc.
- Throttling: control the usage of API. When throttle limit is reached, server returns “429 - Too many requests".

## Functional requirement

- Limit number of requests (Eg. 15 requests per second)
- APIs are accessible through a cluster.

## Non-functional requirement

- High availability.
- Low latency.

## High-level design

- Client <-> web server <-> rate limiter
- Client <-> web server <-> API server
- Web server first asks rate limiter whether request should be served or throttled.

# Design Twitter Search

- Search over all user tweets.

## Capacity estimate

- Assume
    - 1.5B total users, 800M daily active users.
    - 400M tweets per day.
    - Each tweet is 300 bytes.
    - 500M searches per day.
    - Search query contains multiple words.
- Storage: (400M tweets/day) * (300 bytes/tweet) = 120GB/day = 1.38MB/s

## API

- search(api_dev_key, search_terms, max_results_to_return)

## High-level design

- Client <-> app server <-> DB server

# Design Web Crawler

## High-level design

- Pick a URL from unvisited URL list.
- Determine the ID address.
- Download the document.
- Parse document contents to look for new URLs.
- Add new URLs to unvisited URL list.
- Process downloaded document.

# Design Facebook Newsfeed

## Functional requirement

- Feed is generated based on posts that user follows.
- Feed may contain images, videos, texts.
- Supports adding new posts as they arrive.

## Non-functional requirement

- Generates feeds in real-time, with latency less than 2s.

## Capacity estimate

- Assume
    - User has 300 friends.
    - User follows 200 pages.
    - 300M daily active users.
    - User fetches timeline 5 times a day.
- Traffic
    - 1.5B feed requests per day or 17500 requests per second.
- Storage

## API

- getUserFeed(api_dev_key, user_id)

## DB

# Design Yelp

# Design Uber 

# Design Ticketmaster